In [101]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType , DoubleType
from pyspark.sql.functions import ceil , col, split
from pyspark.sql import functions as F
import pandas as  pd



In [11]:
import matplotlib.pyplot as plt

In [3]:
spark = (SparkSession.builder
  .master("local")
  .appName("horror_dataset")
  .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/26 12:07:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [33]:
path = 'raw_data/IMDB Horror movies.csv'
df = spark.read.csv(path, header=True, inferSchema= True)
type(df) 

pyspark.sql.dataframe.DataFrame

In [34]:
df.show(20)

+--------------------+--------------------+------------+---------------+------------+-------------+--------------+--------------------+--------------------+---------+--------------------+-----------------+
|               Title|              Genres|Release Date|Release Country|Movie Rating|Review Rating|Movie Run Time|                Plot|                Cast| Language|   Filming Locations|           Budget|
+--------------------+--------------------+------------+---------------+------------+-------------+--------------+--------------------+--------------------+---------+--------------------+-----------------+
|          Gut (2012)| Drama| Horror| T...|   26-Oct-12|            USA|        null|          3.9|        91 min|Directed by Elias...|Jason Vail|Nichol...|  English|       New York, USA|             null|
|The Haunting of M...|              Horror|   13-Jan-17|            USA|        null|         null|          null|Directed by Jake ...|Nicola Fiore|Brin...|  English|          

In [35]:
df = df.withColumnRenamed('Release Date','Release_Date') \
        .withColumnRenamed('Release Country','Release_Country') \
        .withColumnRenamed('Movie Rating','Movie_Rating') \
        .withColumnRenamed('Review Rating', 'Review_Rating') \
        .withColumnRenamed('Movie Run Time', 'Run_Time') \
        .withColumnRenamed('Filming Locations' , 'Filming_Locations')
df.show(20)

+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+
|               Title|              Genres|Release_Date|Release_Country|Movie_Rating|Review_Rating|Run_Time|                Plot|                Cast| Language|   Filming_Locations|           Budget|
+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+
|          Gut (2012)| Drama| Horror| T...|   26-Oct-12|            USA|        null|          3.9|  91 min|Directed by Elias...|Jason Vail|Nichol...|  English|       New York, USA|             null|
|The Haunting of M...|              Horror|   13-Jan-17|            USA|        null|         null|    null|Directed by Jake ...|Nicola Fiore|Brin...|  English|                null|         $30,000 |


In [39]:
df = df.na.fill(value='Not_Rank', subset=['Movie_Rating'])
df = df.na.fill(value='0', subset=['Review_Rating'])
df = df.na.fill(value='unknown', subset=['Filming_Locations'])
df = df.na.fill (value=0, subset=['Review_Rating'])
df = df.na.fill (value='0', subset=['Run_Time'])
df = df.na.fill (value='0', subset=['Budget'])

In [42]:
df.show(20)

+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+
|               Title|              Genres|Release_Date|Release_Country|Movie_Rating|Review_Rating|Run_Time|                Plot|                Cast| Language|   Filming_Locations|           Budget|
+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+
|          Gut (2012)| Drama| Horror| T...|   26-Oct-12|            USA|    Not_Rank|          3.9|  91 min|Directed by Elias...|Jason Vail|Nichol...|  English|       New York, USA|                0|
|The Haunting of M...|              Horror|   13-Jan-17|            USA|    Not_Rank|          0.0|       0|Directed by Jake ...|Nicola Fiore|Brin...|  English|             unknown|         $30,000 |


In [99]:
df2 = df.select("*", ceil(col('Review_Rating')).alias('Last_Ranking'))
df2.show(20)



+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+------------+
|               Title|              Genres|Release_Date|Release_Country|Movie_Rating|Review_Rating|Run_Time|                Plot|                Cast| Language|   Filming_Locations|           Budget|Last_Ranking|
+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+------------+
|          Gut (2012)| Drama| Horror| T...|   26-Oct-12|            USA|    Not_Rank|          3.9|  91 min|Directed by Elias...|Jason Vail|Nichol...|  English|       New York, USA|                0|           4|
|The Haunting of M...|              Horror|   13-Jan-17|            USA|    Not_Rank|          0.0|       0|Directed by Jake ...|Nicola Fiore|Brin..

In [112]:
df2 = df2.withColumn('Languages_list',split(col('Language'),'\\|'))
leng_list = df2.selectExpr('max(size(Languages_list))').collect()[0][0]
for i in range(leng_list):
    df3 = df2.withColumn(f'leng_{i+1}', col('Languages_list')[i])
df3.show()
#df3.printSchema()





+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+------------+--------------+-------+
|               Title|              Genres|Release_Date|Release_Country|Movie_Rating|Review_Rating|Run_Time|                Plot|                Cast| Language|   Filming_Locations|           Budget|Last_Ranking|Languages_list|leng_15|
+--------------------+--------------------+------------+---------------+------------+-------------+--------+--------------------+--------------------+---------+--------------------+-----------------+------------+--------------+-------+
|          Gut (2012)| Drama| Horror| T...|   26-Oct-12|            USA|    Not_Rank|          3.9|  91 min|Directed by Elias...|Jason Vail|Nichol...|  English|       New York, USA|                0|           4|     [English]|   null|
|The Haunting of M...|              Horror|   13-Jan-17|

In [78]:
display(pdf)

,languageArray
0,"[E, n, g, l, i, s, h, ]"
1,"[E, n, g, l, i, s, h, ]"
2,"[E, n, g, l, i, s, h, ]"
3,"[E, n, g, l, i, s, h, ]"
4,"[I, t, a, l, i, a, n, ]"
...,...
3323,"[E, n, g, l, i, s, h, ]"
3324,"[E, n, g, l, i, s, h, ]"
3325,"[E, n, g, l, i, s, h, ]"
3326,"[E, n, g, l, i, s, h, ]"


In [61]:
pdf = df2.toPandas()
plt.bar(pdf['Language'], pdf['Last_ranking'])
plt.xlabel = 'Lenguages'
plt.ylab = 'Ranking'
plt.title('Grafico Horror Movies')
plt.show()



KeyError: 'Last_ranking'